In [1]:
!nvidia-smi 

Sun Apr 19 07:48:12 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:3B:00.0 Off |                  N/A |
| 46%   59C    P0    55W / 250W |      0MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:5E:00.0 Off |                  N/A |
| 41%   57C    P0    68W / 250W |      0MiB / 11019MiB |      1%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [2]:
'''# For clean

import os
import fnmatch
 
INTERNAL_DATA_PATH = 'cropped_coco/'
# Get a list of all files in directory
for rootDir, subdirs, filenames in os.walk(INTERNAL_DATA_PATH):
    # Find the files that matches the given patterm
    for filename in fnmatch.filter(filenames, '.*'):
        try:
            #print(filename)
            os.remove(os.path.join(rootDir, filename))
        except OSError:
            print("Error while deleting file")
    for subdir in fnmatch.filter(subdirs, '.*'):
        try:
            #print(filename)
            os.remove(os.path.join(rootDir, subdir))
        except OSError:
            print("Error while deleting file")'''

'# For clean\n\nimport os\nimport fnmatch\n \nINTERNAL_DATA_PATH = \'cropped_coco/\'\n# Get a list of all files in directory\nfor rootDir, subdirs, filenames in os.walk(INTERNAL_DATA_PATH):\n    # Find the files that matches the given patterm\n    for filename in fnmatch.filter(filenames, \'.*\'):\n        try:\n            #print(filename)\n            os.remove(os.path.join(rootDir, filename))\n        except OSError:\n            print("Error while deleting file")\n    for subdir in fnmatch.filter(subdirs, \'.*\'):\n        try:\n            #print(filename)\n            os.remove(os.path.join(rootDir, subdir))\n        except OSError:\n            print("Error while deleting file")'

In [3]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision
import torch
from torch.utils.data import Dataset, DataLoader
import os
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm


########################################################################
batch_size = 128
########################################################################


In [4]:
DATASET_PATH = 'sampled_cropped_coco'
TRAIN_PATH = DATASET_PATH + '/train/100'
TEST_PATH = DATASET_PATH + '/val/100'

In [5]:
#img_size = [729,243,81,27,9]  

default_transform = transforms.Compose(
    [transforms.Resize((224,224)),
    transforms.ToTensor()])

trainCocoDataset = datasets.ImageFolder(root=TRAIN_PATH,transform = default_transform)
testCocoDataset = datasets.ImageFolder(root=TEST_PATH,transform = default_transform)

trainloader = torch.utils.data.DataLoader(trainCocoDataset,
                                          batch_size = batch_size, 
                                          shuffle=True,
                                         num_workers=4)

testloader = torch.utils.data.DataLoader(testCocoDataset,
                                          batch_size = batch_size, 
                                          shuffle=False,
                                        num_workers=4)


In [6]:
'''trainCocoDataset.class_to_idx'''

'trainCocoDataset.class_to_idx'

In [7]:
from SCResnet import SCResNet50,ResNet50
from torchvision.models import resnet50
#torchvision.models.resnet50(pretrained=False, progress=True, **kwargs)


In [8]:
from torch.nn.parallel.data_parallel import DataParallel
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.distributed as dist

os.environ['MASTER_ADDR'] = '127.0.0.1'
os.environ['MASTER_PORT'] = '29500'
dist.init_process_group(backend='nccl',rank=0,world_size=1)

In [9]:
device = 'cuda:0'
num_classes = 58

model1 = resnet50(num_classes=num_classes).to(device)
if torch.cuda.device_count() > 1:
    model1 = DDP(model1,device_ids = [0,1])

In [10]:
import os
from torchvision import transforms

SIZED_VAL_PATH = 'sampled_cropped_coco/val'

val_gen_arr = []
for i in tqdm(range(10,110,10)):
    transform = transforms.Compose(
        [
        transforms.Resize((224,224)),
        transforms.ToTensor()])

    testCocoDataset_val = datasets.ImageFolder(root=os.path.join(SIZED_VAL_PATH,str(i)),transform = transform)

    testloader_val = torch.utils.data.DataLoader(testCocoDataset_val,
                                              batch_size = batch_size, 
                                              shuffle=False,
                                            num_workers=4)
    val_gen_arr.append(testloader_val)


100%|██████████| 10/10 [00:00<00:00, 11.03it/s]


In [11]:
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

learning_rate = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model1.parameters(), lr=learning_rate)

In [12]:
TEST_RESULT_PATH = 'testresults/'
WEIGHTS_PATH = 'weights/'

TASK_NAME = 'COCO_Resnet_50_sample'
DATE = 'apr19'

In [13]:
import torch.optim as optim
import sys

trn_loss_list = []
val_loss_list = []
acc_list = []
acc_list_aug = []
num_batches = len(trainloader)
for epoch in range(20):
    running_loss = 0.0
    model1.train()
    #device = 'cuda:0'
    for i,images in tqdm(enumerate(trainloader)):
        optimizer.zero_grad()
        c1 = images[0].to(device)
        c6 = images[1].to(device)
        output = model1(c1)
        loss = criterion(output, c6)
        loss.backward()    # calc gradients
        optimizer.step()   # update gradients
        running_loss += loss.item()
    
        if (i+1) % 6500 == 0:
            model1.eval()
            with torch.no_grad(): # very very very very important!!
                val_loss = 0.0
                class_correct = list(0. for i in range(num_classes))
                class_total = list(0. for i in range(num_classes))
                correct = 0
                total = 0
                for j,val in enumerate(testloader):
                    v1 = val[0].to(device)
                    val_labels = val[1].to(device)
                    val_output = model1(v1)
                    v_loss = criterion(val_output, val_labels)
                    val_loss += v_loss
                    _, predicted = torch.max(val_output, 1)
                    c = (predicted == val_labels).squeeze()
                    total += val_labels.size(0)
                    correct += (predicted == val_labels).sum().item()
                    for i in range(len(val_labels)):
                        val_label = val_labels[i]
                        class_correct[val_label] += c[i].item()
                        class_total[val_label] += 1

                print("epoch:",str(epoch),str(i)," batch")
                for i in range(num_classes):
                    if class_total[i]==0:
                        print('class_total = 0',class_correct,class_total)
                    else:
                        print('Accuracy of %5s : %2d %%' % (i, 100 * class_correct[i] / class_total[i]))

            print("epoch: {}/{} | step: {}/{} | trn loss: {:.4f} | val loss: {:.4f}".format(
                epoch+1, 20, i+1, num_batches, running_loss / len(trainloader), val_loss / len(testloader)
            ))        
            print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

            trn_loss_list.append(running_loss/1875)
            val_loss_list.append(val_loss/len(testloader))
            running_loss = 0.0

            temp_acc = []
            for testloader_val in tqdm(val_gen_arr):
                correct = 0
                total = 0
                class_correct = list(0. for i in range(num_classes))
                class_total = list(0. for i in range(num_classes))
                with torch.no_grad():
                    for images in testloader_val:
                        c1 = images[0].to(device)
                        val_labels = images[1].to(device)
                        outputs = model1(c1)
                        _, predicted = torch.max(outputs.data, 1)
                        c = (predicted == val_labels).squeeze()
                        total += val_labels.size(0)
                        correct += (predicted == val_labels).sum().item()
                #print(total,correct,end='')

                print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))
                temp_acc.append(100 * correct / total)
                #print(temp_acc)
            acc_list.append(temp_acc)

            import csv

            csvfile = open(TEST_RESULT_PATH+'testresult_'+TASK_NAME+'_'+DATE+'.csv','w',newline="")

            csvwriter = csv.writer(csvfile)
            for row in acc_list:
                csvwriter.writerow(row)

            csvfile.close()

            torch.save(model1.state_dict(),WEIGHTS_PATH+TASK_NAME+'_'+DATE+'.pt')

            model1.train()
            
    model1.eval()
    with torch.no_grad(): # very very very very important!!!
        val_loss = 0.0
        class_correct = list(0. for i in range(num_classes))
        class_total = list(0. for i in range(num_classes))
        correct = 0
        total = 0
        for j,val in enumerate(testloader):
            v1 = val[0].to(device)
            val_labels = val[1].to(device)
            val_output = model1(v1)
            v_loss = criterion(val_output, val_labels)
            val_loss += v_loss
            _, predicted = torch.max(val_output, 1)
            c = (predicted == val_labels).squeeze()
            total += val_labels.size(0)
            correct += (predicted == val_labels).sum().item()
            for i in range(len(val_labels)):
                val_label = val_labels[i]
                class_correct[val_label] += c[i].item()
                class_total[val_label] += 1

        print("epoch:",str(epoch),str(i)," batch")
        for i in range(num_classes):
            if class_total[i]==0:
                print('class_total = 0',class_correct,class_total)
            else:
                print('Accuracy of %5s : %2d %%' % (i, 100 * class_correct[i] / class_total[i]))

    print("epoch: {}/{} | step: {}/{} | trn loss: {:.4f} | val loss: {:.4f}".format(
        epoch+1, 20, i+1, num_batches, running_loss / len(trainloader), val_loss / len(testloader)
    ))        
    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

    trn_loss_list.append(running_loss/1875)
    val_loss_list.append(val_loss/len(testloader))
    running_loss = 0.0

    temp_acc = []
    for testloader_val in tqdm(val_gen_arr):
        correct = 0
        total = 0
        class_correct = list(0. for i in range(num_classes))
        class_total = list(0. for i in range(num_classes))
        with torch.no_grad():
            for images in testloader_val:
                c1 = images[0].to(device)
                val_labels = images[1].to(device)
                outputs = model1(c1)
                _, predicted = torch.max(outputs.data, 1)
                c = (predicted == val_labels).squeeze()
                total += val_labels.size(0)
                correct += (predicted == val_labels).sum().item()
        #print(total,correct,end='')

        print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))
        temp_acc.append(100 * correct / total)
        #print(temp_acc)
    acc_list.append(temp_acc)

    import csv

    csvfile = open(TEST_RESULT_PATH+'testresult_'+TASK_NAME+'_'+DATE+'.csv','w',newline="")

    csvwriter = csv.writer(csvfile)
    for row in acc_list:
        csvwriter.writerow(row)

    csvfile.close()

    torch.save(model1.state_dict(),WEIGHTS_PATH+TASK_NAME+'_'+DATE+'.pt')

    model1.train()
    '''if epoch == 9:
        optimizer = optim.Adam(model1.parameters(), lr=0.0001)
        print('lr is changed to 0.0001')'''
        
    #optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)'''
    

1360it [07:39,  2.96it/s]
  0%|          | 0/10 [00:00<?, ?it/s]

epoch: 0 67  batch
Accuracy of     0 :  1 %
Accuracy of     1 : 10 %
Accuracy of     2 : 11 %
Accuracy of     3 :  0 %
Accuracy of     4 :  4 %
Accuracy of     5 : 19 %
Accuracy of     6 : 26 %
Accuracy of     7 :  9 %
Accuracy of     8 : 12 %
Accuracy of     9 :  7 %
Accuracy of    10 : 69 %
Accuracy of    11 : 21 %
Accuracy of    12 : 33 %
Accuracy of    13 : 11 %
Accuracy of    14 : 27 %
Accuracy of    15 : 13 %
Accuracy of    16 : 37 %
Accuracy of    17 :  2 %
Accuracy of    18 : 49 %
Accuracy of    19 : 68 %
Accuracy of    20 : 12 %
Accuracy of    21 : 27 %
Accuracy of    22 : 33 %
Accuracy of    23 : 16 %
Accuracy of    24 : 14 %
Accuracy of    25 :  3 %
Accuracy of    26 :  5 %
Accuracy of    27 : 51 %
Accuracy of    28 :  0 %
Accuracy of    29 : 41 %
Accuracy of    30 :  1 %
Accuracy of    31 : 12 %
Accuracy of    32 : 24 %
Accuracy of    33 : 25 %
Accuracy of    34 :  1 %
Accuracy of    35 : 47 %
Accuracy of    36 : 56 %
Accuracy of    37 : 49 %
Accuracy of    38 : 27 %
Accura

 10%|█         | 1/10 [00:28<04:12, 28.11s/it]

Accuracy of the network on the 10000 test images: 5 %


 20%|██        | 2/10 [00:56<03:45, 28.16s/it]

...

Accuracy of the network on the 10000 test images: 32 %


 90%|█████████ | 9/10 [04:10<00:27, 27.94s/it]

Accuracy of the network on the 10000 test images: 42 %


100%|██████████| 10/10 [04:38<00:00, 27.86s/it]

Accuracy of the network on the 10000 test images: 53 %


In [ ]:
images[1]

In [ ]:
for i,images in tqdm(enumerate(trainloader)):
    print(images[1].shape)
    break

# TRAIN AND TEST ENDS
code below won't work

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(3,64,kernel_size=7,stride=2,padding=3,bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace = True)
        self.maxpool = nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
        
        self.prepool1 = nn.Upsample(scale_factor=2, mode='bilinear')
        self.prepool2 = nn.AvgPool2d(2,2)
        
        #self.sc1 = SCModule1(3,64)
        self.rp1 = ResPoolModule(256)
        self.rp2 = ResPoolModule(512)
        self.rp3 = ResPoolModule(1024)
        self.rp4 = ResPoolModule(2048)
        
        #self.relu = nn.ReLU(inplace = True)
        
        self.conv2_1 = SCBottleneck(64,256,downsample=True)
        self.conv2_2 = SCBottleneck(256,256)
        self.conv2_3 = SCBottleneck(256,256)
        
        self.conv3_1 = SCBottleneck(256,512,downsample=True)
        self.conv3_2 = SCBottleneck(512,512)
        self.conv3_3 = SCBottleneck(512,512)
        self.conv3_4 = SCBottleneck(512,512)
        
        self.conv4_1 = SCBottleneck(512,1024,downsample=True)
        self.conv4_2 = SCBottleneck(1024,1024)
        self.conv4_3 = SCBottleneck(1024,1024)
        self.conv4_4 = SCBottleneck(1024,1024)
        self.conv4_5 = SCBottleneck(1024,1024)
        self.conv4_6 = SCBottleneck(1024,1024)
        
        self.conv5_1 = SCBottleneck(1024,2048,downsample=True)
        self.conv5_2 = SCBottleneck(2048,2048)
        self.conv5_3 = SCBottleneck(2048,2048)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(2048,10)

    def forward(self, x):
        save = []
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        save.append(x)
        
        #print(x.size())
            
        x = self.conv2_1([self.prepool1(x),x,self.prepool2(x)])
        save.append(x[0])
        save.append(x[1])
        save.append(x[2])
        x = self.conv2_2(x)
        save.append(x[0])
        save.append(x[1])
        save.append(x[2])
        x = self.conv2_3(x)
        save.append(x[0])
        save.append(x[1])
        save.append(x[2])
        
        x = self.rp1(x)
        save.append(x)
        
        #print(x.size())
        
        x = self.conv3_1([self.prepool1(x),x,self.prepool2(x)])
        save.append(x[0])
        save.append(x[1])
        save.append(x[2])
        x = self.conv3_2(x)
        save.append(x[0])
        save.append(x[1])
        save.append(x[2])
        x = self.conv3_3(x)
        save.append(x[0])
        save.append(x[1])
        save.append(x[2])
        x = self.conv3_4(x)
        save.append(x[0])
        save.append(x[1])
        save.append(x[2])
        
        x = self.rp2(x)
        save.append(x)
        
        x = self.conv4_1([self.prepool1(x),x,self.prepool2(x)])
        save.append(x[0])
        save.append(x[1])
        save.append(x[2])
        x = self.conv4_2(x)
        save.append(x[0])
        save.append(x[1])
        save.append(x[2])
        x = self.conv4_3(x)
        save.append(x[0])
        save.append(x[1])
        save.append(x[2])
        x = self.conv4_4(x)
        save.append(x[0])
        save.append(x[1])
        save.append(x[2])
        x = self.conv4_5(x)
        save.append(x[0])
        save.append(x[1])
        save.append(x[2])
        x = self.conv4_6(x)
        save.append(x[0])
        save.append(x[1])
        save.append(x[2])
        
        x = self.rp3(x)
        save.append(x)
        
        x = self.conv5_1([self.prepool1(x),x,self.prepool2(x)])
        save.append(x[0])
        save.append(x[1])
        save.append(x[2])
        x = self.conv5_2(x)
        save.append(x[0])
        save.append(x[1])
        save.append(x[2])
        x = self.conv5_3(x)
        save.append(x[0])
        save.append(x[1])
        save.append(x[2])
        
        x = self.rp4(x)
        save.append(x)
        
        x = self.avgpool(x)
        x = torch.flatten(x,1)
        x = self.fc(x)
        
        return x,save
        
        

In [ ]:
model1 = Net().to('cuda:0')
if torch.cuda.device_count() > 1:
    model1 = DDP(model1,device_ids = [0])
'''    
model2 = Net().to('cuda:0')
if torch.cuda.device_count() > 1:
    model2 = DDP(model2,device_ids = [2])'''

In [ ]:
model1.load_state_dict(torch.load('ResNet50_allsc_half_mar11.pt'))
#model2.load_state_dict(torch.load('ResNet50_scrp2_feb19.pt'))

In [ ]:
model1.eval()
#model2.eval()

In [ ]:
from PIL import Image

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307, 0.1307, 0.1307), (0.3081, 0.3081, 0.3081))])

SIZED_VAL_PATH = 'mnist_sized'
l2_img_test_arr = []
for i in range(1,113):
    img_path = os.path.join(SIZED_VAL_PATH,str(i),'9','1622.png')
    img = Image.open(img_path)
    img = transform(img)
    img = torch.unsqueeze(img,0)
    img.to(device)
    l2_img_test_arr.append(img)

In [ ]:
!mkdir res50_allsc_l2

In [ ]:
name = ['1st conv',\
        'sc2_1_1(256)','sc2_1_2(256)','sc2_1_3(256)',\
        'sc2_2_1(256)','sc2_2_2(256)','sc2_2_3(256)',\
        'sc2_3_1(256)','sc2_3_2(256)','sc2_3_3(256)',\
        'rp2',\
        'sc3_1_1(512)','sc3_1_2(512)','sc3_1_3(512)',\
        'sc3_2_1(512)','sc3_2_2(512)','sc3_2_3(512)',\
        'sc3_3_1(512)','sc3_3_2(512)','sc3_3_3(512)',\
        'sc3_4_1(512)','sc3_4_2(512)','sc3_4_3(512)',\
        'rp3',\
        'sc4_1_1(1024)','sc4_1_2(1024)','sc4_1_3(1024)',\
        'sc4_2_1(1024)','sc4_2_2(1024)','sc4_2_3(1024)',\
        'sc4_3_1(1024)','sc4_3_2(1024)','sc4_3_3(1024)',\
        'sc4_4_1(1024)','sc4_4_2(1024)','sc4_4_3(1024)',\
        'sc4_5_1(1024)','sc4_5_2(1024)','sc4_5_3(1024)',\
        'sc4_6_1(1024)','sc4_6_2(1024)','sc4_6_3(1024)',\
        'rp4',\
        'sc5_1_1(2048)','sc5_1_2(2048)','sc5_1_3(2048)',\
        'sc5_2_1(2048)','sc5_2_2(2048)','sc5_2_3(2048)',\
        'sc5_3_1(2048)','sc5_3_2(2048)','sc5_3_3(2048)',\
        'rp5']

size = 1
for img in tqdm(l2_img_test_arr):
    with torch.no_grad():
        outputs,save = model1(img)
        _, predicted = torch.max(outputs.data, 1)
    print(predicted)
    
    j = 0
    plt.figure(figsize=(24,12))
    plt.suptitle('resnet L2ALL SC half image size:'+str(size)+'\n'+'predicted :'+str(predicted.cpu().numpy()[0])+' ground truth:9')
    #plt.suptitle('resnet L2 ALL layer SC half')
    for act in save:
        dist = []
        index = np.arange(act[0].size()[0])
        for i in range(act[0].size()[0]):
            dist.append(torch.norm(act[0][i],p=2).cpu().detach().numpy())
        plt.subplot(6,9,j+1)
        plt.title(name[j])
        plt.bar(index,dist)
        j = j+1
        plt.gca().axes.get_xaxis().set_visible(False)
    plt.savefig('res50_allsc_l2/l2_'+str(size)+'.png')
    plt.show()
    size = size+1

In [ ]:
import cv2

frame_array = []
for i in range(1,113):
    filename='res50_allsc_l2/l2_'+str(i)+'.png'
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    
    #inserting the frames into an image array
    frame_array.append(img)
    
out = cv2.VideoWriter('res50_allsc_l2/allsc_half_l2_mar15.mp4',cv2.VideoWriter_fourcc(*'MP4V'), 5, size)
for i in range(len(frame_array)):
    # writing to a image array
    out.write(frame_array[i])
out.release()

In [ ]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.detach().numpy()
    #plt.imshow(np.transpose(npimg, (1, 2, 0)))
    return np.transpose(npimg, (1, 2, 0))

In [ ]:
import matplotlib.cm as cm

def show_gradient(gradient):
    #gradient = gradient.cpu().numpy().transpose(1, 2, 0)
    gradient = gradient.cpu().numpy().transpose(1,2,0)
    gradient -= gradient.min()
    gradient /= gradient.max()
    gradient *= 255.0
    plt.imshow(np.squeeze(np.uint8(gradient),axis=2))
    plt.show()

def show_gradcam(gcam, raw_image, paper_cmap=False,show = True):
    gcam = gcam.cpu().numpy()
    cmap = cm.jet(gcam)[..., :3]*255.0#
    if paper_cmap:
        alpha = gcam[..., None]
        gcam = alpha * cmap + (1 - alpha) * raw_image
    else:
        gcam = (cmap.astype(np.float) + raw_image.astype(np.float)) / 2
        #gcam = (cmap.astype(np.float)*raw_image.astype(np.float))
    if show:
        plt.imshow(gcam.astype(np.uint8))
        plt.show()
        return 0
    else:
        return gcam.astype(np.uint8)

In [ ]:
!mkdir 'gradcam_2x_respool_soloconv'

In [ ]:
from sklearn.metrics import confusion_matrix

confusion = []

temp_acc = []
device = 'cuda:0'
for testloader in tqdm(val_gen_arr):
    correct = 0
    total = 0
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    confusion.append([])
    with torch.no_grad():
        for images in testloader:
            c1 = images[0].to(device)
            val_labels = images[1].to(device)
            outputs = model1(c1)
            _, predicted = torch.max(outputs.data, 1)
            #print(predicted)
            c = (predicted == val_labels).squeeze()
            total += val_labels.size(0)
            correct += (predicted == val_labels).sum().item()
            
            confusion[-1].append(confusion_matrix(val_labels.cpu().numpy(),predicted.cpu().numpy(),labels=[0,1,2,3,4,5,6,7,8,9]))
    #print(total,correct,end='')

    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))
    temp_acc.append(100 * correct / total)
    
#print(confusion)


In [ ]:
confusion_sum = []

for i in range(len(confusion)):
    confusion_sum.append(confusion[i][0])
    for j in range(1,len(confusion[i])):
        confusion_sum[i] += confusion[i][j]
        
print(confusion_sum[0])

In [ ]:
!mkdir confusion_2x_respool_soloconv

In [ ]:
for size in range(0,112):
    fig = plt.figure(figsize=(12,12))
    pred_name = ['pred '+str(x) for x in range(0,10)]
    actual_name = ['true '+str(x) for x in range(0,10)]
    ax = fig.add_subplot(111)
    cax = ax.matshow(confusion_sum[size],cmap = plt.cm.Greens)
    fig.colorbar(cax)
    ticks = np.arange(0,10,1)

    ind_array = np.arange(0, 10, 1)
    x, y = np.meshgrid(ind_array, ind_array)

    for x_val, y_val in zip(x.flatten(), y.flatten()):
        c = str(confusion_sum[size][y_val][x_val])
        ax.text(x_val, y_val, c, va='center', ha='center')

    ax.set_xticks(ticks)
    ax.set_yticks(ticks)
    ax.set_xticklabels(pred_name)
    ax.set_yticklabels(actual_name)
    ax.set_title('size : '+str(size+1)+' acc : '+str(temp_acc[size]))
    #plt.matshow(confusion_sum[0])
    plt.savefig('confusion_2x_respool_soloconv/confusion'+str(size)+'.png')
    plt.show()

In [ ]:
import cv2

frame_array = []
for i in range(0,112):
    filename='confusion_2x_respool_soloconv/confusion' + str(i)+'.png'
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    
    #inserting the frames into an image array
    frame_array.append(img)
    
out = cv2.VideoWriter('confusion_2x_respool_soloconv/confusion_feb13.mp4',cv2.VideoWriter_fourcc(*'MP4V'), 5, size)
for i in range(len(frame_array)):
    # writing to a image array
    out.write(frame_array[i])
out.release()

In [ ]:
import os


SIZED_VAL_PATH = 'mnist_sized'
new_path = []
for i in range(1,113):
    new_path.append([os.path.join(SIZED_VAL_PATH,str(i))])

#new_path.sort(key=lambda x:int(x[0].split('/')[1]))

gradcam_gen_arr = []
for i in tqdm(range(0,len(new_path))):
    transform = transforms.Compose(
        [
        transforms.ToTensor(),
        transforms.Normalize((0.1307, 0.1307, 0.1307), (0.3081, 0.3081, 0.3081))])

    testMnistDataset = SizedMnistDataset(new_path[i],transform)

    testloader = torch.utils.data.DataLoader(testMnistDataset,
                                              batch_size = batch_size, 
                                              shuffle=False,
                                            num_workers=8)
    gradcam_gen_arr.append(testloader)
#
#val_gen_arr
accuracy = []

In [ ]:
from gradcam import GradCAM
from gradcam import GuidedBackPropagation

#model1.eval()
#model2.eval()
gcam = GradCAM(model=model1)
gbp = GuidedBackPropagation(model=model2)

In [ ]:
for name, module in model1.named_modules():
    print(name)

In [ ]:
for i in gcam.fmap_pool.keys():
    print(i)

In [ ]:
from matplotlib import gridspec

gridspec.GridSpec(3,6)

imsize = 1
for testloader in tqdm(gradcam_gen_arr):
    for images in testloader:
        v1 = images[0].to(device)
        val_labels = images[1].to(device)
        break

    _ = gcam.forward(v1)
    probs,ids = gbp.forward(v1)

    gcam.backward(ids=ids[:,[0]])
    gbp.backward(ids=ids[:,[0]])

    grad_img = []

    for j in range(len(v1)):
        grad_img.append([])
        for name, module in model1.named_modules():
            #print(name)
            if name == '' or name == 'module' or name == 'module.fc1' or name == 'module.fc2' or name == 'module.fc3' or name == 'module.prepool1' or name == 'module.prepool2':
                continue
            regions = gcam.generate(target_layer=name)
            gradients = gbp.generate()
            #print("\t#{}: {} ({:.5f})".format(j, ids[j, 0], probs[j, 0]))
            #print(regions[j,0])
            if name == 'module.sc1' or name == 'module.sc2' or name == 'module.sc3':
                grad_img[j].append(show_gradcam(gcam=regions[0][j,0],raw_image=imshow(v1[j].cpu()),show=False))
                grad_img[j].append(show_gradcam(gcam=regions[1][j,0],raw_image=imshow(v1[j].cpu()),show=False))
                grad_img[j].append(show_gradcam(gcam=regions[2][j,0],raw_image=imshow(v1[j].cpu()),show=False))
            if name == 'module.rp1':
                grad_img[j].append(show_gradcam(gcam=regions[j,0],raw_image=imshow(v1[j].cpu()),show=False))

    img_num = [0,1,2,3,4,5,15,16,17,18,19,20,31,32,33,34]
    col_num = [3,3,3,1]
    title = ['sc1 (L1,L2,L3)','sc2(L1,L2,L3)','sc3(L1,L2,L3)','rp1']
    size = 112
    
    scale = 3./size
    plt.figure(figsize=(18,9))
    plt.suptitle('SC_CNN_2x_respool_gradcam'+'_size:'+str(imsize)+'_acc:'+str(temp_acc[imsize-1])+'\n predicted:{} ({:.5f})'.format(ids[0, 0], probs[0, 0]))
    num = 0
    for i in range(len(col_num)):
        display_grid = np.zeros((112,112*col_num[i],3)).astype(np.uint8)
        for row in range(col_num[i]):
            display_grid[:size,row*size : (row+1)*size,:3] = grad_img[0][num]
            num = num+1
        if i == 3:
            plt.subplot2grid((3,6), (i//2,(i%2)*3),colspan = 1,rowspan=1)
        else:
            plt.subplot2grid((3,6), (i//2,(i%2)*3),colspan = 3,rowspan=1)
        plt.axis('off')
        plt.title(title[i])
        plt.grid(False)
        plt.imshow(display_grid,aspect='auto')
    plt.savefig('gradcam_2x_respool/pltgrad'+str(imsize)+'.png')
    plt.show()
    imsize = imsize+1

In [ ]:
v1.size()

In [ ]:
frame_array = []
for i in range(1,113):
    filename='gradcam_2x/pltgrad' + str(i)+'.png'
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    
    #inserting the frames into an image array
    frame_array.append(img)
    
out = cv2.VideoWriter('gradcam_2x_respool/gradcam_ordered_feb13.mp4',cv2.VideoWriter_fourcc(*'MP4V'), 5, size)
for i in range(len(frame_array)):
    # writing to a image array
    out.write(frame_array[i])
out.release()

In [ ]:
for i in gcam.fmap_pool.keys():
    print(i)

In [ ]:
num=0

#plt.figure(figsize=(12,5))
plt.matshow([gcam.fmap_pool['module.fc1'].cpu().numpy()[num]])
plt.matshow([gcam.fmap_pool['module.fc2'].cpu().numpy()[num]])
plt.matshow([gcam.fmap_pool['module.fc3'].cpu().numpy()[num]])

plt.show()

In [ ]:
i=0
print(gcam.fmap_pool['module.prepool1'].cpu().numpy()[0][i].var())
plt.imshow(gcam.fmap_pool['module.prepool2'].cpu().numpy()[0][i])
plt.show()

In [ ]:
gcam.fmap_pool['module.sc1_1'].cpu().numpy()[0][0]

In [ ]:
num = 1
#print(gcam.fmap_pool['module.sc1_1'].cpu().numpy()[0][num])
plt.figure(figsize=(16,12))
for i in range(6):
    plt.subplot(3,6,i+1)
    plt.imshow(gcam.fmap_pool['module.sc1_1'].cpu().numpy()[0][i])
for i in range(6):
    plt.subplot(3,6,i+1+6)
    plt.imshow(gcam.fmap_pool['module.sc1_2'].cpu().numpy()[0][i])
for i in range(6):
    plt.subplot(3,6,i+1+12)
    plt.imshow(gcam.fmap_pool['module.sc1_3'].cpu().numpy()[0][i])
plt.show()

In [ ]:
num = 1
#print(gcam.fmap_pool['module.sc1_1'].cpu().numpy()[0][num])
plt.figure(figsize=(16,5))
for i in range(18):
    plt.subplot(3,18,i+1)
    plt.imshow(gcam.fmap_pool['module.sc2_1'].cpu().numpy()[0][i])
for i in range(18):
    plt.subplot(3,18,i+1+18)
    plt.imshow(gcam.fmap_pool['module.sc2_2'].cpu().numpy()[0][i])
for i in range(18):
    plt.subplot(3,18,i+1+36)
    plt.imshow(gcam.fmap_pool['module.sc2_3'].cpu().numpy()[0][i])
plt.show()

In [ ]:
num = 1
#print(gcam.fmap_pool['module.sc1_1'].cpu().numpy()[0][num])
plt.figure(figsize=(16,5))
for i in range(30):
    plt.subplot(3,30,i+1)
    plt.imshow(gcam.fmap_pool['module.sc3_1'].cpu().numpy()[0][i])
for i in range(30):
    plt.subplot(3,30,i+1+30)
    plt.imshow(gcam.fmap_pool['module.sc3_2'].cpu().numpy()[0][i])
for i in range(30):
    plt.subplot(3,30,i+1+60)
    plt.imshow(gcam.fmap_pool['module.sc3_3'].cpu().numpy()[0][i])
plt.show()

In [ ]:
num = 1
#print(gcam.fmap_pool['module.sc1_1'].cpu().numpy()[0][num])
plt.figure(figsize=(16,5))
for i in range(30):
    plt.subplot(3,10,i+1)
    plt.imshow(gcam.fmap_pool['module.rp1'].cpu().numpy()[0][i])
plt.show()

In [ ]:
gcam.fmap_pool['module.sc1_1'].cpu().numpy().shape

In [ ]:
num = 1
conv_list = ['module.sc1.conv1','module.sc1.conv2','module.sc1.conv3']
conv_num = 0
#print(gcam.fmap_pool[conv_list[conv_num]].cpu().numpy()[0][num])
for i in range(2):
    plt.subplot(1,2,i+1)
    plt.imshow(gcam.fmap_pool[conv_list[conv_num]].cpu().numpy()[0][i])
plt.show()

In [ ]:
num = 1
conv_list = ['module.sc1.conv1','module.sc1.conv2','module.sc1.conv3']
conv_num = 1
#print(gcam.fmap_pool[conv_list[conv_num]].cpu().numpy()[0][num])
for i in range(2):
    plt.subplot(1,2,i+1)
    plt.imshow(gcam.fmap_pool[conv_list[conv_num]].cpu().numpy()[0][i])
plt.show()

In [ ]:
num = 1
conv_list = ['module.sc1.conv1','module.sc1.conv2','module.sc1.conv3']
conv_num = 2
#print(gcam.fmap_pool[conv_list[conv_num]].cpu().numpy()[0][num])
for i in range(2):
    plt.subplot(1,2,i+1)
    plt.imshow(gcam.fmap_pool[conv_list[conv_num]].cpu().numpy()[0][i])
plt.show()

In [ ]:
model1.module.sc1.conv1.weight.shape

In [ ]:
for i in range(3):
    print(model1.module.sc1.conv1.weight[0][i].cpu().detach().numpy())
    plt.imshow(model1.module.sc1.conv1.weight[0][i].cpu().detach().numpy())#.shape
    plt.show()

In [ ]:
for i in range(3):
    print(model1.module.sc1.conv2.weight[0][i].cpu().detach().numpy())
    plt.imshow(model1.module.sc1.conv2.weight[0][i].cpu().detach().numpy())#.shape
    plt.show()

In [ ]:
for i in range(3):
    print(model1.module.sc1.conv3.weight[0][i].cpu().detach().numpy())
    plt.imshow(model1.module.sc1.conv3.weight[0][i].cpu().detach().numpy())#.shape
    plt.show()